In [3]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import leafmap
import polyline
import requests

# Load the restaurant data
df = pd.read_csv('restaurants1.csv')
df = df[['longitude', 'latitude']]

# Convert to GeoDataFrame
geometry = gpd.GeoSeries.from_xy(df.longitude, df.latitude, crs="EPSG:4326")
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

print(gdf.head())


   longitude   latitude                   geometry
0  77.070819  28.590664  POINT (77.07082 28.59066)
1  77.071500  28.591582   POINT (77.0715 28.59158)
2  77.072300  28.592700    POINT (77.0723 28.5927)
3  77.073400  28.593800    POINT (77.0734 28.5938)


In [4]:
import requests
import polyline

def get_route_coordinates(df):
    coordinates = ';'.join([f"{row['longitude']},{row['latitude']}" for index, row in df.iterrows()])
    return coordinates

def fetch_route(coordinates):
    url = f"https://router.project-osrm.org/route/v1/driving/{coordinates}?overview=full"
    response = requests.get(url)
    data = response.json()
    return data['routes'][0]['geometry']

def decode_polyline(encoded_polyline):
    decoded_polyline = polyline.decode(encoded_polyline, 5)
    return [(lat, lon) for lon, lat in decoded_polyline]  # Reverse lat/lon order

# Example usage
coordinates = get_route_coordinates(df)
encoded_polyline = fetch_route(coordinates)
decoded_route = decode_polyline(encoded_polyline)


In [5]:
import keplergl

# Create Kepler.gl Map
map_1 = keplergl.KeplerGl()

# Add restaurant data to the map
map_1.add_data(data=df, name="Restaurants")

# Add route data to the map
route_df = pd.DataFrame(decoded_route, columns=['latitude', 'longitude'])
map_1.add_data(data=route_df, name="Route")

# Show the map
map_1


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'Restaurants': {'index': [0, 1, 2, 3], 'columns': ['longitude', 'latitude'], 'data': [[77.07081…

In [6]:
print("Encoded Polyline:", encoded_polyline)
print("Decoded Route:", decoded_route)


Encoded Polyline: cbomDq|kuMNJJTR~@BVGNS`@FDFDR_@FMBI?O?OI]k@yB[ESEC?KCKEUCQCKAc@Ee@IMGKGOUE[@e@XoBHWFIHEHAZDb@HTDVJJDFFP?B?B?H?PDJ}@LmA@Ek@YqAUgASy@GOA]vCIDM?_Eu@i@SERCJ`B\HBBF?JYzAe@pCEFE@UEa@Mq@[AB?DABALcBo@OIIIQQMQAMCY@]b@mDNuALBB?FBM|@?HBFl@J
Decoded Route: [(77.07097, 28.59058), (77.07091, 28.5905), (77.0708, 28.59044), (77.07048, 28.59034), (77.07036, 28.59032), (77.07028, 28.59036), (77.07011, 28.59046), (77.07008, 28.59042), (77.07005, 28.59038), (77.07021, 28.59028), (77.07028, 28.59024), (77.07033, 28.59022), (77.07041, 28.59022), (77.07049, 28.59022), (77.07064, 28.59027), (77.07125, 28.59049), (77.07128, 28.59063), (77.07131, 28.59073), (77.07131, 28.59075), (77.07133, 28.59081), (77.07136, 28.59087), (77.07138, 28.59098), (77.0714, 28.59107), (77.07141, 28.59113), (77.07144, 28.59131), (77.07149, 28.5915), (77.07153, 28.59157), (77.07157, 28.59163), (77.07168, 28.59171), (77.07182, 28.59174), (77.07201, 28.59173), (77.07257, 28.5916), (77.07269, 28.59155), (77.07274, 28.5

In [8]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import leafmap
import polyline
import requests

# Load the restaurant data
df = pd.read_csv('restaurants1.csv')
df = df[['longitude', 'latitude']]

# Convert to GeoDataFrame
geometry = gpd.GeoSeries.from_xy(df.longitude, df.latitude, crs="EPSG:4326")
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# Print GeoDataFrame
print(gdf.head())

# Generate route coordinates
def get_route_coordinates(df):
    coordinates = ';'.join([f"{row['longitude']},{row['latitude']}" for index, row in df.iterrows()])
    return coordinates

# Fetch route from OSRM
def fetch_route(coordinates):
    url = f"https://router.project-osrm.org/route/v1/driving/{coordinates}?overview=full"
    print("OSRM URL:", url)  # Print the URL being requested
    
    response = requests.get(url)
    print("OSRM Response:", response.json())  # Print the response JSON
    
    data = response.json()
    return data['routes'][0]['geometry']

# Decode polyline
def decode_polyline(encoded_polyline):
    decoded_polyline = polyline.decode(encoded_polyline, 5)
    return [(lon, lat) for lat, lon in decoded_polyline]

# Get route coordinates
coordinates = get_route_coordinates(df)

# Fetch and decode the route
encoded_polyline = fetch_route(coordinates)
decoded_route = decode_polyline(encoded_polyline)

# Create Kepler.gl map
import keplergl

map_1 = keplergl.KeplerGl()

# Add restaurant data to the map
map_1.add_data(data=df, name="Restaurants")

# Convert route data to GeoJSON
route_geojson = {
    "type": "FeatureCollection",
    "features": [{
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": decoded_route
        },
        "properties": {}
    }]
}

# Add route data to the map
map_1.add_data(data=route_geojson, name="Route")

# Show the map
map_1


   longitude   latitude                   geometry
0  77.070819  28.590664  POINT (77.07082 28.59066)
1  77.071500  28.591582   POINT (77.0715 28.59158)
2  77.072300  28.592700    POINT (77.0723 28.5927)
3  77.073400  28.593800    POINT (77.0734 28.5938)
OSRM URL: https://router.project-osrm.org/route/v1/driving/77.070819,28.590664;77.0715,28.591582;77.0723,28.5927;77.0734,28.5938?overview=full
OSRM Response: {'code': 'Ok', 'routes': [{'geometry': 'cbomDq|kuMNJJTR~@BVGNS`@FDFDR_@FMBI?O?OI]k@yB[ESEC?KCKEUCQCKAc@Ee@IMGKGOUE[@e@XoBHWFIHEHAZDb@HTDVJJDFFP?B?B?H?PDJ}@LmA@Ek@YqAUgASy@GOA]vCIDM?_Eu@i@SERCJ`B\\HBBF?JYzAe@pCEFE@UEa@Mq@[AB?DABALcBo@OIIIQQMQAMCY@]b@mDNuALBB?FBM|@?HBFl@J', 'legs': [{'steps': [], 'summary': '', 'weight': 3332.2, 'duration': 56.5, 'distance': 360.8}, {'steps': [], 'summary': '', 'weight': 3411.6, 'duration': 140.7, 'distance': 839.5}, {'steps': [], 'summary': '', 'weight': 3362.1, 'duration': 87.6, 'distance': 511.1}], 'weight_name': 'routability', 'weight': 10105.9,

KeplerGl(data={'Restaurants': {'index': [0, 1, 2, 3], 'columns': ['longitude', 'latitude'], 'data': [[77.07081…